In [1]:
import pandas as pd

logs_df = pd.read_csv("my_selected_logs.csv")
labels_df = pd.read_csv("my_selected_blocks.csv")

print(logs_df.shape)
print(labels_df.shape)

logs_df.head(), labels_df.head()


(1333, 10)
(100, 3)


(   LineId   Date    Time  Pid Level                 Component  \
 0     500  81109  203532  150  INFO  dfs.DataNode$DataXceiver   
 1     571  81109  203532   29  INFO          dfs.FSNamesystem   
 2     657  81109  203533  150  INFO  dfs.DataNode$DataXceiver   
 3     686  81109  203533  166  INFO  dfs.DataNode$DataXceiver   
 4    1016  81109  203535  150  INFO  dfs.DataNode$DataXceiver   
 
                                              Content EventId  \
 0  Receiving block blk_6309418393114897475 src: /...      E5   
 1  BLOCK* NameSystem.allocateBlock: /user/root/ra...     E22   
 2  Receiving block blk_6309418393114897475 src: /...      E5   
 3  Receiving block blk_6309418393114897475 src: /...      E5   
 4  Receiving block blk_-8588230903310885315 src: ...      E5   
 
                               EventTemplate                   BlockId  
 0  Receiving block <*> src: /<*> dest: /<*>   blk_6309418393114897475  
 1       BLOCK* NameSystem.allocateBlock:<*>   blk_6309418393114

In [5]:
labels_df["Label"] = labels_df["Label"].map({"Normal": 0, "Anomaly": 1})


In [6]:
ecm = logs_df.pivot_table(
    index="BlockId",
    columns="EventId",
    aggfunc="size",
    fill_value=0
)

print("Event Count Matrix shape:", ecm.shape)
ecm.head()



Event Count Matrix shape: (100, 13)


EventId,E11,E13,E16,E18,E2,E21,E22,E25,E26,E27,E5,E6,E9
BlockId,,,,,,,,,,,,,
blk_-1051183647010524801,3,0,0,0,0,0,1,0,3,0,3,0,3
blk_-1559690858566389454,3,0,0,0,0,0,1,0,3,0,3,0,3
blk_-1738283557460552416,3,0,0,0,0,0,1,0,3,0,3,0,3
blk_-1954051922809122540,3,0,0,0,0,0,1,0,3,0,3,0,3
blk_-2276466771376002171,3,0,0,0,0,0,1,0,3,0,3,0,3


In [7]:
dataset = ecm.merge(labels_df, on="BlockId", how="inner")

print("Merged dataset:", dataset.shape)
dataset.head()



Merged dataset: (100, 16)


,BlockId,E11,E13,E16,E18,E2,E21,E22,E25,E26,E27,E5,E6,E9,NumEvents,Label
0,blk_-1051183647010524801,3,0,0,0,0,0,1,0,3,0,3,0,3,13,NaN
1,blk_-1559690858566389454,3,0,0,0,0,0,1,0,3,0,3,0,3,13,NaN
2,blk_-1738283557460552416,3,0,0,0,0,0,1,0,3,0,3,0,3,13,NaN
3,blk_-1954051922809122540,3,0,0,0,0,0,1,0,3,0,3,0,3,13,NaN
4,blk_-2276466771376002171,3,0,0,0,0,0,1,0,3,0,3,0,3,13,NaN


In [8]:
X = dataset.drop(columns=["BlockId", "Label"], errors="ignore")
X = X.select_dtypes(include=['number'])
y = dataset["Label"]

print("PCA Input shape:", X.shape)
print("Dtypes:\n", X.dtypes)


PCA Input shape: (100, 14)
Dtypes:
 E11          int64
E13          int64
E16          int64
E18          int64
E2           int64
E21          int64
E22          int64
E25          int64
E26          int64
E27          int64
E5           int64
E6           int64
E9           int64
NumEvents    int64
dtype: object


In [11]:
# Remove rows with missing labels
dataset = dataset.dropna(subset=["Label"])

# Convert label column to int again (sometimes gets float type)
dataset["Label"] = dataset["Label"].astype(int)

print("After removing NaN labels:", dataset.shape)
print("Label distribution:\n", dataset["Label"].value_counts())


After removing NaN labels: (0, 16)
Label distribution:
 Series([], Name: count, dtype: int64)


In [12]:
# Remove missing labels
dataset = dataset.dropna(subset=["Label"])
dataset["Label"] = dataset["Label"].astype(int)

# PCA input must be numeric only
X = dataset.drop(columns=["BlockId", "Label"], errors="ignore")
X = X.select_dtypes(include=['number'])

y = dataset["Label"]


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)
X_recon = pca.inverse_transform(X_pca)

# Error
errors = np.mean((X_scaled - X_recon)**2, axis=1)
threshold = np.median(errors)
y_pred = (errors > threshold).astype(int)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("\n===== PCA BASELINE RESULTS =====")
print("Accuracy :", accuracy_score(y, y_pred))
print("Precision:", precision_score(y, y_pred))
print("Recall   :", recall_score(y, y_pred))
print("F1 Score :", f1_score(y, y_pred))


attempt 1

In [20]:
import pandas as pd

blocks = pd.read_csv("my_selected_blocks.csv")
logs = pd.read_csv("my_selected_logs.csv")

print("Blocks file:")
print(blocks.head(10))
print("\nUnique labels:", blocks["Label"].unique())

print("\nLogs file:")
print(logs.head(10))


Blocks file:
                    BlockId  NumEvents  Label
0   blk_8716244237763720958         13      1
1   blk_6291404884140655419         13      1
2   blk_8751446418791214913         13      1
3   blk_4961970861863065608         13      1
4   blk_3888635850409849568         14      1
5  blk_-3842070622043972712         16      1
6  blk_-2437547284244111497         13      1
7   blk_-423065043106849758         13      1
8   blk_8491971867438278117         13      1
9   blk_8464056021668582878         13      1

Unique labels: [1 0]

Logs file:
   LineId   Date    Time  Pid Level                 Component  \
0     500  81109  203532  150  INFO  dfs.DataNode$DataXceiver   
1     571  81109  203532   29  INFO          dfs.FSNamesystem   
2     657  81109  203533  150  INFO  dfs.DataNode$DataXceiver   
3     686  81109  203533  166  INFO  dfs.DataNode$DataXceiver   
4    1016  81109  203535  150  INFO  dfs.DataNode$DataXceiver   
5    1187  81109  203535   35  INFO          dfs.FSNamesy

In [21]:
import pandas as pd

# Load both datasets
logs = pd.read_csv("my_selected_logs.csv")
blocks = pd.read_csv("my_selected_blocks.csv")

# Build event count matrix (pivot)
event_matrix = logs.pivot_table(
    index="BlockId",
    columns="EventId",
    aggfunc="size",
    fill_value=0
).reset_index()

# Merge labels
event_matrix = event_matrix.merge(
    blocks[["BlockId", "Label"]],
    on="BlockId",
    how="left"
)

print("Event matrix shape:", event_matrix.shape)
event_matrix.head()



Event matrix shape: (100, 15)


,BlockId,E11,E13,E16,E18,E2,E21,E22,E25,E26,E27,E5,E6,E9,Label
0,blk_-1051183647010524801,3,0,0,0,0,0,1,0,3,0,3,0,3,0
1,blk_-1559690858566389454,3,0,0,0,0,0,1,0,3,0,3,0,3,0
2,blk_-1738283557460552416,3,0,0,0,0,0,1,0,3,0,3,0,3,0
3,blk_-1954051922809122540,3,0,0,0,0,0,1,0,3,0,3,0,3,0
4,blk_-2276466771376002171,3,0,0,0,0,0,1,0,3,0,3,0,3,0


In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load event matrix (already created)
df = event_matrix.copy()

# Separate features & labels
X = df.drop(columns=["BlockId", "Label"])
y = df["Label"].astype(int)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA: keep 95% variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)
X_recon = pca.inverse_transform(X_pca)

# Reconstruction error per block
errors = np.mean((X_scaled - X_recon)**2, axis=1)

# Threshold = median reconstruction error
threshold = np.median(errors)
y_pred = (errors > threshold).astype(int)

# Evaluation
print("\n===== PCA BASELINE RESULTS =====")
print(f"Accuracy : {accuracy_score(y, y_pred):.4f}")
print(f"Precision: {precision_score(y, y_pred):.4f}")
print(f"Recall   : {recall_score(y, y_pred):.4f}")
print(f"F1 Score : {f1_score(y, y_pred):.4f}")

# Attach predictions back to dataframe (optional)
df["recon_error"] = errors
df["pred_label"] = y_pred

df.head()



===== PCA BASELINE RESULTS =====
Accuracy : 0.6900
Precision: 0.4857
Recall   : 0.5667
F1 Score : 0.5231


,BlockId,E11,E13,E16,E18,E2,E21,E22,E25,E26,E27,E5,E6,E9,Label,recon_error,pred_label
0,blk_-1051183647010524801,3,0,0,0,0,0,1,0,3,0,3,0,3,0,6.148161e-33,0
1,blk_-1559690858566389454,3,0,0,0,0,0,1,0,3,0,3,0,3,0,3.059710e-31,1
2,blk_-1738283557460552416,3,0,0,0,0,0,1,0,3,0,3,0,3,0,5.000114e-30,1
3,blk_-1954051922809122540,3,0,0,0,0,0,1,0,3,0,3,0,3,0,1.991115e-32,1
4,blk_-2276466771376002171,3,0,0,0,0,0,1,0,3,0,3,0,3,0,4.324453e-32,1


In [23]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Feature matrix (remove non-numeric)
X = df.drop(columns=["BlockId", "Label", "recon_error", "pred_label"])
y = df["Label"].astype(int)

# Contamination = fraction of anomalies
cont = y.sum() / len(y)

iso = IsolationForest(contamination=cont, random_state=42)
y_pred_iso = iso.fit_predict(X)

# Convert from {-1,1} → {1,0}
y_pred_iso = np.where(y_pred_iso == -1, 1, 0)

print("\n===== ISOLATION FOREST RESULTS =====")
print("Accuracy :", accuracy_score(y, y_pred_iso))
print("Precision:", precision_score(y, y_pred_iso))
print("Recall   :", recall_score(y, y_pred_iso))
print("F1 Score :", f1_score(y, y_pred_iso))



===== ISOLATION FOREST RESULTS =====
Accuracy : 0.69
Precision: 0.4666666666666667
Recall   : 0.23333333333333334
F1 Score : 0.3111111111111111


In [24]:
from sklearn.svm import OneClassSVM

# Feature matrix
X = df.drop(columns=["BlockId", "Label", "recon_error", "pred_label"])
y = df["Label"].astype(int)

# ν ~ anomaly fraction
nu = y.sum() / len(y)

svm = OneClassSVM(kernel="rbf", gamma="auto", nu=nu)
y_pred_svm = svm.fit_predict(X)

# Convert {-1,1} → {1,0}
y_pred_svm = np.where(y_pred_svm == -1, 1, 0)

print("\n===== ONE-CLASS SVM RESULTS =====")
print("Accuracy :", accuracy_score(y, y_pred_svm))
print("Precision:", precision_score(y, y_pred_svm))
print("Recall   :", recall_score(y, y_pred_svm))
print("F1 Score :", f1_score(y, y_pred_svm))



===== ONE-CLASS SVM RESULTS =====
Accuracy : 0.69
Precision: 0.4666666666666667
Recall   : 0.23333333333333334
F1 Score : 0.3111111111111111
